In [1]:
from NeuroPy import NeuroPy
import time
import keyboard
import numpy as np
import pandas as pd
import datetime
import random
import sys
import pylsl

#from sklearn.neural_network import MLPClassifier


In [2]:
#Connect to Headset
MW = NeuroPy();
MW.start();

In [3]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get('https://en.wikipedia.org/wiki/Artificial_neural_network')

In [ ]:
# MAIN LOOP, READ ATTN/MED AND SCROLL PAGE

scroll_pos = 0
run = 1
h_max = driver.execute_script("return document.body.scrollHeight")

atn_cutoff = 70
med_cutoff = 70
scroll_step = 20
scroll_up = 0
print 'Starting Blink sample gathering'
while(run == 1):
    
    atn_list = []
    med_list = []
    for i in range(0,20):
        atn_list.append(MW.attention)
        med_list.append(MW.meditation)
        time.sleep(0.2)
    avg_atn = ( sum(atn_list) / len(atn_list) )
    avg_med = ( sum(med_list) / len(med_list) )
    
    print('average attention: ',avg_atn)
    print('average meditation: ',avg_med)
    if(avg_atn > atn_cutoff):
        
        if(scroll_pos + scroll_step < h_max ):
            scroll_pos = scroll_pos + scroll_step 
            driver.execute_script("window.scrollTo(0, %d)" % scroll_pos)
            scroll_up = 0
    
    elif(avg_med > avg_atn and avg_med > med_cutoff):
        if(scroll_up == 0):
            if(scroll_pos - scroll_step < 0 ):
                scroll_pos = scroll_pos - scroll_step 
                driver.execute_script("window.scrollTo(0, %d)" % scroll_pos)
                scroll_up = 1
        else:
            pass
    else:
        pass
        
    
    
    
    


Starting Blink sample gathering
('average attention: ', 78)
('average meditation: ', 43)
('average attention: ', 71)
('average meditation: ', 36)
('average attention: ', 95)
('average meditation: ', 55)
('average attention: ', 99)
('average meditation: ', 75)
('average attention: ', 95)
('average meditation: ', 73)
('average attention: ', 77)
('average meditation: ', 71)
('average attention: ', 65)
('average meditation: ', 66)
('average attention: ', 99)
('average meditation: ', 59)
('average attention: ', 96)
('average meditation: ', 59)
('average attention: ', 94)
('average meditation: ', 40)
('average attention: ', 93)
('average meditation: ', 48)
('average attention: ', 97)
('average meditation: ', 49)
('average attention: ', 100)
('average meditation: ', 44)
('average attention: ', 100)
('average meditation: ', 35)
('average attention: ', 100)
('average meditation: ', 46)


In [ ]:
#DEPRECATED#

#Connect to openVibe LSL stream
streams = pylsl.resolve_stream('type','signal')
inlet = pylsl.stream_inlet(streams[0])

sample = pylsl.vectorf()


In [ ]:
#DEPRECATED#

#UPDATED BLINK GATHERING

Headset_Reading = []
Sample_List = [[]]
Sample_Array = np.array(np.zeros(900))
Data_Array = np.array([0,0,0,0,0,0,0,0,0])
trainLabel = []

run = 1
num_Blinks = 0
num_NoBlinks = 0

print(inlet.pull_sample(Headset_Reading))
print(inlet.pull_sample(Headset_Reading))
print(inlet.pull_sample(Headset_Reading))
print(inlet.pull_sample(Headset_Reading))
print(inlet.pull_sample(Headset_Reading))
print(inlet.pull_sample(Headset_Reading))




print 'Starting Blink sample gathering'
while(run == 1):
    
    inp = raw_input('enter [b] to gather next Blink sample, [n] for next Non-Blink sample, or [q] to exit')
    if(inp == 'q'):
        run = 0
        print('Quiting, collected %s Blinks and %s noneblinks',num_Blinks,num_NoBlinks)
    elif(inp == 'b'):
        num_Blinks = num_Blinks + 1
        for t in range(0,10):
            print('get ready - %s', 10-t)
            time.sleep(0.1)
            
        Blink_Sample = []
        for smpl in range(0,100):
            #Headset_Reading = [MW.delta,MW.theta,MW.lowAlpha,MW.highAlpha,MW.lowBeta,MW.highBeta,MW.lowGamma,MW.midGamma]
            inlet.pull_sample(Headset_Reading)
            Blink_Sample.extend(Headset_Reading)
            #Blink_Sample.extend([random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50)])
            Data_Array = np.vstack([Data_Array,Headset_Reading])
            print(Headset_Reading)
            time.sleep(0.01)

        Sample_Array = np.vstack([Sample_Array,Blink_Sample])
        Sample_List.append(Blink_Sample) 
        trainLabel.extend([1])
        print('Collected Blink, num blink: %s, num none: %s',num_Blinks,num_NoBlinks)
    elif(inp == 'n'):
        num_NoBlinks = num_NoBlinks + 1
        for t in range(0,10):
            print('get ready - %s', 10-t)
            time.sleep(0.1)
            
        NoBlink_Sample = []
        for smpl in range(0,100):
            #Headset_Reading = [MW.delta,MW.theta,MW.lowAlpha,MW.highAlpha,MW.lowBeta,MW.highBeta,MW.lowGamma,MW.midGamma]
            inlet.pull_sample(Headset_Reading)
            NoBlink_Sample.extend(Headset_Reading)
            #NoBlink_Sample.extend([random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50),random.randint(1,50)])
            Data_Array = np.vstack([Data_Array,Headset_Reading])
            time.sleep(0.01)
        
        Sample_Array = np.vstack([Sample_Array,NoBlink_Sample])
        Sample_List.append(NoBlink_Sample) 
        trainLabel.extend([0])
        print('Collected non-Blink, num blink: %s, num none: %s',num_Blinks,num_NoBlinks)
    else:
        pass
    
    

In [ ]:
print(len(Sample_List))
print(len(trainLabel))

Sample_List_All = Sample_List[1:]
Sample_List_Training = Sample_List_All[:30]
Labels_Training = trainLabel[:30]
trainLabel

In [ ]:
#DEPRECATED#

#    Neural Net

from sklearn.neural_network import MLPClassifier

blinkNet = MLPClassifier()



In [ ]:
#DEPRECATED#

#Save dataset to file
import csv
with open('SampleData.csv','wb') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(Data_Array)
csvFile.close()

In [ ]:
#DEPRECATED#


#Train/Test Split

TotalData = trainData_Blink.append(trainData_NoBlink)
TotalData = TotalData.join(trainLabel)


#Wont work, need groups of 100 not randomizes

totalData = TotalData.sample(frac = 1)

TrainSet = totalData.iloc[:75,:]
TrainLabel = 
TestSet = totalData.iloc[75:,:]


In [ ]:
#DEPRECATED#

#Training 

#for i in range(1,50):
    #smplData = trainData_Blink.iloc[ i*100:((i+1)*100)-1 , : ]
    #smplData = Blink_List[i]
    #blinkNet.fit(smplData,)

Sample_List_Train = SampleList[:30,:]    
training    

print(np.array(Blink_List).shape)
labels = [1]*50
blinkNet.fit(BL, labels)

In [ ]:
#DEPRECATED#

#Testing, 
correct = 0
for i in range(0,75):
    smplData = TestData_Blink.iloc[ i*100:((i+1)*100)-1 , : ]
    if(blinkNet.predict() == TestLabel[i]):
        correct = correct + 1;